<img src="./01/logo-insa.png" style="float:right; max-width: 60px; display: inline" alt="INSA" /></a>

# Sizing of the power converter on static criteria
*Written by Marc Budinger, INSA Toulouse, France*

The purpose of this notebook is to structure the sizing procedure of the inverter and its cooling system.

In general, the establishment of a sizing procedure involves the resolution of the following 3 problems:
* a set of equations sub-constrained by the addition of a design variable in the optimization problem;
* an over-constrained variable by adding a design variable (multiplier) and the transfer of the excess equation(s) in the constrained part of the optimization problem;
* an algebraic loop by the use of a simplify equation weighted by a multiplying coefficient and a constraint representing the initial equation.



Where possible:
* the design variables must take the form of a normalized variable around 1 (oversize coef. for example) or easily bounded to facilitate the work of the optimization algorithm.
* the constraints must take the form of inequality and not of equality more difficult to manage. The optimization of the objective (for example the total mass) will certainly force some (active) constraints to come to an end. 

## 1. Specification and equations

### 1.1. Inputs and specifications

The purpose of the sizing will be to minimize the mass of the cooling system. This mass depends mainly on the losses of the converter. In this notebook we will take into account the average value of losses in transistors and diodes.


In [1]:
Loss1D=91 # [W] Mean power loss of one diode of the inverter
Loss1T=139 # [W] Mean power loss of one transisotr of the inverter


### 1.2. Design assumptions

In [2]:
Nfan=1 # [-] Number of fans

## Thermal sizing scenario

### Design graph

> **Question:** Represent the sizing problem as an undirected graph. Use this graph to wisely choose the orientation of the calculations and the variables of the optimization problem.

![Graphic Thermal Design](08/IGBT_design_graph.png)




### Optimization problem: synthesis

> Exercice: synthesis here the design variables, the constraints which have to be add to the optimization problem.

![PseudoCode](08/PseudoCodeInductor.png)



### Sizing code and optimization

> **Exercice:** propose a sizing code and a optimization loop for the cooling system.

Before performing the overall optimization, each team member can test part of the code on a subset of the system in order to work collaboratively.

In [3]:
from math import pi, sqrt

# Coolants 
π = pi

'''Coolant flow characteristics (40°C)'''
λ_c  = 0.393              #[W/m.K] thermal conductivity
ρ_c  = 1030               #[kg/m3] density
cp_c = 3690               #[J/Kg.K] heat mass capacity
μ_c  = 0.00299            #[Pa.s] dynamic viscosity

'''Air characteristics (27°C)'''
T_in_a = 27            #[°C] input temperature
λ_a  = 0.0262          #[W/m.K] thermal conductivity
ρ_a  = 1.177           #[kg/m3] density 
cp_a = 1006            #[J/Kg.K] heat mass capacity
μ_a  = 1.85e-5         #[Pa.s] dynamic viscosity

T_air=40 # [K] Inlet air temperature

In [12]:
import numpy as np
import scipy
import scipy.optimize
import pandas as pd
import timeit
import ipywidgets as widgets
from ipywidgets import interactive

pd.options.display.float_format = '{:,.2f}'.format

def Sizing(param, arg):
    import math as m

    # Inputs
    # --


        
    # Power module
    # --


    # Cold Plate
    # --


    # Pipes
    # --

    
    # Heat exchanger
    # --
    

    # Pump
    # --

 
    
    # Fan
    # --

    
    # Total mass (to optimize)
    # --
    Total_mass = Nfan*Fan_Mass + Pump_Mass + HE_Mass + Pipes_mass # [kg]

    # Objective and Constraints sum up
    # ---
    objective = Total_mass
    
    constraints = []
    
    # Objective and contraints
    if arg=='Obj':
        return objective
    
    elif arg=='Prt':
    # the data to print a defined into a Pandas dataframe
        col_names = ['Type', 'Name', 'Value', 'Unit', 'Comment']

        df = pd.DataFrame()


        df = df.append([{'Type': 'Specification', 'Name': 'Total loss', 'Value': LossTotal, 'Unit': '[W]', 'Comment': ''}])[col_names]
        df = df.append([{'Type': 'Objective', 'Name': 'Total mass', 'Value': objective, 'Unit': '[kg]', 'Comment': 'Global mass'}])[col_names]
        df = df.append([{'Type': 'Constraints', 'Name': 'HE_loss-LossTotal', 'Value': LossHE-LossTotal, 'Unit': '[W]', 'Comment': ''}])[col_names]
        df = df.append([{'Type': 'Constraints', 'Name': 'T_ColdPlate-T_water_cold', 'Value': T_ColdPlate-T_water_cold, 'Unit': '[K]', 'Comment': ''}])[col_names]
        #df = df.append([{'Type': 'Constraints', 'Name': 'Const 3', 'Value': constraints[2], 'Unit': '[m]', 'Comment': 'Developped length'}])[col_names]
        df = df.append([{'Type': 'Variables', 'Name': 'Junction temperature', 'Value': param[0], 'Unit': '[K]', 'Comment': 'T_j'}])[col_names]
        df = df.append([{'Type': 'Variables', 'Name': 'Water temperature', 'Value': param[1], 'Unit': '[K]', 'Comment': 'T water cold'}])[col_names]
        df = df.append([{'Type': 'Variables', 'Name': 'Air speed', 'Value': param[2], 'Unit': '[m/s]', 'Comment': 'V air'}])[col_names]
        df = df.append([{'Type': 'Variables', 'Name': 'Cr', 'Value': param[3], 'Unit': '[-]', 'Comment': 'HE'}])[col_names]
        df = df.append([{'Type': 'Variables', 'Name': 'NTU', 'Value': param[4], 'Unit': '[-]', 'Comment': 'HE'}])[col_names]        
        df = df.append([{'Type': 'Variables', 'Name': 'RD_pipes', 'Value': param[5], 'Unit': '[-]', 'Comment': 'HE'}])[col_names]        
        df = df.append([{'Type': 'Pump', 'Name': 'Mass', 'Value': Pump_Mass, 'Unit': '[kg]', 'Comment': 'Mass'}])[col_names]
        df = df.append([{'Type': 'Pump', 'Name': 'Pressure', 'Value': dP_water_total, 'Unit': '[Pa]', 'Comment': 'Pressure'}])[col_names]
        df = df.append([{'Type': 'Pump', 'Name': 'Flow Rate', 'Value': Q_ColdPlate, 'Unit': '[l/min]', 'Comment': 'Flow rate'}])[col_names]
        df = df.append([{'Type': 'Fan', 'Name': 'Mass', 'Value': Fan_Mass, 'Unit': '[kg]', 'Comment': 'Mass'}])[col_names]
        df = df.append([{'Type': 'Fan', 'Name': 'Flow Rate', 'Value': Q_air*3600, 'Unit': '[m^3/h]', 'Comment': 'Flow rate'}])[col_names]
        df = df.append([{'Type': 'Fan', 'Name': 'Delta Pressure', 'Value': Fan_dP , 'Unit': '[Pa]', 'Comment': ''}])[col_names]
        df = df.append([{'Type': 'Heat Exchanger', 'Name': 'Mass', 'Value': HE_Mass, 'Unit': '[kg]', 'Comment': 'Mass'}])[col_names]
        df = df.append([{'Type': 'Heat Exchanger', 'Name': 'Efficacity', 'Value': HE_Eff, 'Unit': '[-]', 'Comment': ''}])[col_names]        
        df = df.append([{'Type': 'Heat Exchanger', 'Name': 'UA', 'Value': UA, 'Unit': '[-]', 'Comment': ''}])[col_names]        
        df = df.append([{'Type': 'Heat Exchanger', 'Name': 'Surface', 'Value': S_air, 'Unit': '[m^2]', 'Comment': 'S_air'}])[col_names]        
        df = df.append([{'Type': 'Heat Exchanger', 'Name': 'L', 'Value': HE_L, 'Unit': '[m]', 'Comment': ''}])[col_names]        
        df = df.append([{'Type': 'Heat Exchanger', 'Name': 'H', 'Value': HE_H, 'Unit': '[m]', 'Comment': ''}])[col_names]    
        df = df.append([{'Type': 'Heat Exchanger', 'Name': 'W', 'Value': HE_W, 'Unit': '[m]', 'Comment': 'Thickness'}])[col_names]    
        df = df.append([{'Type': 'Heat Exchanger', 'Name': 'Air speed', 'Value': v_air, 'Unit': '[m/s]', 'Comment': ''}])[col_names]    
        df = df.append([{'Type': 'Cold Plate', 'Name': 'Flow rate', 'Value': Q_ColdPlate, 'Unit': '[l/min]', 'Comment': 'Flow rate'}])[col_names]
        df = df.append([{'Type': 'Cold Plate', 'Name': 'Thermal resistance', 'Value': Rth_ColdPlate*1000, 'Unit': '[mK/W]', 'Comment': 'Rth_ColdPlate'}])[col_names]
        df = df.append([{'Type': 'Cold Plate', 'Name': 'Temperature ColdPlate', 'Value': T_ColdPlate, 'Unit': '[°C]', 'Comment': 'T_ColdPlate'}])[col_names]
        df = df.append([{'Type': 'Cold Plate', 'Name': 'Temperature Hot water', 'Value': T_water_hot, 'Unit': '[°C]', 'Comment': 'T_water_hot'}])[col_names]
        df = df.append([{'Type': 'Cold Plate', 'Name': 'Pressure drop', 'Value': dP_ColdPlate, 'Unit': '[Pa]', 'Comment': ''}])[col_names]
        df = df.append([{'Type': 'Pipes', 'Name': 'Mass', 'Value': Pipes_mass, 'Unit': '[kg]', 'Comment': 'Mass'}])[col_names]
        df = df.append([{'Type': 'Pipes', 'Name': 'Diameter', 'Value': RD_pipe*D_pipes_ref*1e3 , 'Unit': '[mm]', 'Comment': 'Internal diameter'}])[col_names]
        df = df.append([{'Type': 'Pipes', 'Name': 'Pressure drop', 'Value': dP_pipes, 'Unit': '[Pa]', 'Comment': ''}])[col_names]
        
        # the dataframe is then organised to be printed with interactive widgets
        items = sorted(df['Type'].unique().tolist())
         
        #print(df.to_markdown()) 
        
        def f(Type):
            return df[df['Type']==Type] 
        widgets.interact(f, Type=items)
        return f
                  
    else:
        return constraints




In [13]:
#Variables d'optimisation

# Vector of parameters
parameters = np.array((XX,XX,...))


In [14]:
# Initial characteristics before optimization 
print("-----------------------------------------------")
print("Initial characteristics before optimization :")

Sizing(parameters, 'Prt')
print("-----------------------------------------------")




-----------------------------------------------
Initial characteristics before optimization :


interactive(children=(Dropdown(description='Type', options=('Cold Plate', 'Constraints', 'Fan', 'Heat Exchange…

-----------------------------------------------


In [15]:
# Then we can solve the problem and print of the optimized solution:

# In[70]:


# optimization with SLSQP algorithm
contrainte=lambda x: Sizing(x, 'Const')
objectif=lambda x: Sizing(x, 'Obj')

result = scipy.optimize.fmin_slsqp(func=objectif, x0=parameters, 
                                   bounds=[(XX,XX)...],
                                   f_ieqcons=contrainte, iter=100, acc=1e-8, epsilon=0.0001)


# Final characteristics after optimization 
print("-----------------------------------------------")
print("Final characteristics after optimization :")

Sizing(result, 'Prt')

print("-----------------------------------------------")

Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.2877636849995118
            Iterations: 25
            Function evaluations: 169
            Gradient evaluations: 24
-----------------------------------------------
Final characteristics after optimization :


interactive(children=(Dropdown(description='Type', options=('Cold Plate', 'Constraints', 'Fan', 'Heat Exchange…

-----------------------------------------------
